In [3]:
!pip install matplotlib seaborn pandas numpy
!pip install transformers torch tensorflow datasets
!pip install streamlit gradio  # For deployment

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [4]:
!pip install matplotlib seaborn

In [6]:
# CELL 1: Fixed imports - Minimal setup
print("🚀 Starting Gender Equality Chatbot Project...")

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Try importing transformers with error handling
try:
    import torch
    from transformers import T5ForConditionalGeneration, T5Tokenizer
    print("✅ Transformers imported successfully!")
except Exception as e:
    print(f"❌ Transformers import error: {e}")
    print("Installing required packages...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tf-keras"])
    import torch
    from transformers import T5ForConditionalGeneration, T5Tokenizer

from sklearn.model_selection import train_test_split

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

🚀 Starting Gender Equality Chatbot Project...
✅ Transformers imported successfully!
Using device: cpu


In [ ]:
print("\n Loading your dataset...")

try:
    # Load your CSV file
    df = pd.read_csv(r"C:\Users\HP\OneDrive\Desktop\motherschatbot\women-empowerment-chatbot\dataset\womenchatbot.csv")
    print("✅ Dataset loaded successfully!")
    
    # Display dataset info
    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print("\nFirst few rows:")
    print(df.head())
    
    # Check for missing values
    print(f"\nMissing values per column:")
    print(df.isnull().sum())
    
except Exception as e:
    print(f" Error loading dataset: {e}")
    # Fallback to creating a small dataset for testing
    print("Creating a sample dataset for testing...")
    data = {
        'question': [
            "What is women's empowerment?",
            "What does gender equality mean?",
            "Why is girls' education important?"
        ],
        'answer': [
            "Women's empowerment refers to giving women power and authority to make life decisions.",
            "Gender equality means all individuals have equal rights regardless of gender.",
            "Educating girls leads to better health outcomes and economic growth."
        ],
        'category': ['basic_concepts', 'basic_concepts', 'education'],
        'difficulty': ['beginner', 'beginner', 'intermediate']
    }
    df = pd.DataFrame(data)


📁 Loading your dataset...
✅ Dataset loaded successfully!
Dataset shape: (112, 4)
Columns: ['question', 'answer', 'category', 'difficulty']

First few rows:
                                question  \
0           What is women's empowerment?   
1        What does gender equality mean?   
2     Why is girls' education important?   
3  How can we support women's education?   
4            What is the gender pay gap?   

                                              answer        category  \
0  Women's empowerment refers to the process of g...  basic_concepts   
1  Gender equality means that all individuals, re...  basic_concepts   
2  Educating girls is crucial because it leads to...       education   
3  You can support women's education by donating ...       education   
4  The gender pay gap refers to the difference in...       workplace   

     difficulty  
0      beginner  
1      beginner  
2  intermediate  
3  intermediate  
4  intermediate  

Missing values per column:
question 

In [8]:
print("\n🔍 Exploring dataset...")

print(f"Total Q&A pairs: {len(df)}")
print(f"Categories: {df['category'].unique()}")
print(f"Difficulty levels: {df['difficulty'].unique()}")

# Check distribution
print(f"\nCategory distribution:")
print(df['category'].value_counts())

print(f"\nDifficulty distribution:")
print(df['difficulty'].value_counts())


🔍 Exploring dataset...
Total Q&A pairs: 112
Categories: ['basic_concepts' 'education' 'workplace' 'health_rights' 'data_insights'
 'actionable_advice' 'political_participation' 'global_context'
 'economics' 'technology' 'legal_rights' 'environment' 'community' 'media'
 'family' 'sports' 'inclusion' 'communication' 'humanitarian'
 'arts_culture' 'science' 'organizations' 'youth']
Difficulty levels: ['beginner' 'intermediate' 'advanced']

Category distribution:
category
health_rights              13
economics                  13
workplace                  11
education                   9
basic_concepts              8
technology                  7
political_participation     7
family                      5
arts_culture                4
media                       4
legal_rights                4
communication               3
data_insights               3
community                   3
global_context              3
inclusion                   3
sports                      3
actionable_advic

In [9]:

print("\n🔄 Preprocessing data...")

def preprocess_data(df):
    """Preprocess the dataset for T5 training"""
    
    # Handle missing values
    df = df.dropna(subset=['question', 'answer'])
    
    # Create conversation pairs in T5 format
    df['input_text'] = "question: " + df['question'] + " context: gender equality"
    df['target_text'] = df['answer']
    
    # Basic text cleaning
    df['input_text'] = df['input_text'].str.strip()
    df['target_text'] = df['target_text'].str.strip()
    
    # Remove any empty strings
    df = df[(df['input_text'].str.len() > 0) & (df['target_text'].str.len() > 0)]
    
    return df

df_processed = preprocess_data(df)
print(f"✅ Processed dataset shape: {df_processed.shape}")

# Show sample of processed data
print("\nSample processed data:")
for i in range(min(2, len(df_processed))):
    print(f"Input: {df_processed['input_text'].iloc[i][:80]}...")
    print(f"Target: {df_processed['target_text'].iloc[i][:80]}...")
    print("-" * 50)


🔄 Preprocessing data...
✅ Processed dataset shape: (112, 6)

Sample processed data:
Input: question: What is women's empowerment? context: gender equality...
Target: Women's empowerment refers to the process of giving women the power and authorit...
--------------------------------------------------
Input: question: What does gender equality mean? context: gender equality...
Target: Gender equality means that all individuals, regardless of their gender, have equ...
--------------------------------------------------


In [11]:
# CELL 5: Train-Validation Split (Fixed)
print("\n📊 Splitting data...")

# Check category distribution first
print("Category distribution:")
print(df_processed['category'].value_counts())

# For small datasets or categories with few samples, use simple split without stratification
if len(df_processed) < 50 or (df_processed['category'].value_counts() < 2).any():
    print("Using simple split (dataset too small for stratification)")
    train_df, val_df = train_test_split(
        df_processed, 
        test_size=0.2, 
        random_state=42
    )
else:
    print("Using stratified split")
    train_df, val_df = train_test_split(
        df_processed, 
        test_size=0.2, 
        random_state=42,
        stratify=df_processed['category']
    )

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")

# Show distribution in splits
print("\nTraining set category distribution:")
print(train_df['category'].value_counts())

print("\nValidation set category distribution:")
print(val_df['category'].value_counts())


📊 Splitting data...
Category distribution:
category
health_rights              13
economics                  13
workplace                  11
education                   9
basic_concepts              8
technology                  7
political_participation     7
family                      5
arts_culture                4
media                       4
legal_rights                4
communication               3
data_insights               3
community                   3
global_context              3
inclusion                   3
sports                      3
actionable_advice           2
environment                 2
humanitarian                2
science                     1
organizations               1
youth                       1
Name: count, dtype: int64
Using simple split (dataset too small for stratification)
Training samples: 89
Validation samples: 23

Training set category distribution:
category
health_rights              12
economics                  12
workplace              

In [12]:
print("\n🤖 Loading model and tokenizer...")

try:
    model_name = "t5-small"  # Good for starting - fast and efficient
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    
    print("✅ Model and tokenizer loaded successfully!")
    print(f"Model: {model_name}")
    print(f"Vocabulary size: {tokenizer.vocab_size}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")


🤖 Loading model and tokenizer...
❌ Error loading model: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.



In [16]:
# CELL 6: Initialize Model and Tokenizer
print("\n🤖 Loading model and tokenizer...")

try:
    model_name = "t5-small"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    print("✅ Model and tokenizer loaded successfully!")
except Exception as e:
    print(f"❌ Error: {e}")
    # Let's try a different approach
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    tokenizer = AutoTokenizer.from_pretrained("t5-small")
    model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
    print("✅ Model loaded using Auto classes!")

print(f"Tokenizer type: {type(tokenizer)}")


🤖 Loading model and tokenizer...
❌ Error: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.

✅ Model loaded using Auto classes!
Tokenizer type: <class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>


In [19]:
# CELL 7: Create Datasets
print("\n📚 Creating datasets...")

# First, let's verify our dataframes exist
print(f"train_df shape: {train_df.shape}")
print(f"val_df shape: {val_df.shape}")

class GenderEqualityDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, max_length=256):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_text = str(self.data.iloc[idx]['input_text'])
        target_text = str(self.data.iloc[idx]['target_text'])
        
        # Tokenize inputs
        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Tokenize targets
        target_encoding = self.tokenizer(
            target_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }

# Create datasets
train_dataset = GenderEqualityDataset(train_df, tokenizer)
val_dataset = GenderEqualityDataset(val_df, tokenizer)

print("✅ Datasets created successfully!")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")


📚 Creating datasets...
train_df shape: (89, 6)
val_df shape: (23, 6)
✅ Datasets created successfully!
Train dataset size: 89
Val dataset size: 23


In [1]:
# CELL: Fix Keras compatibility and setup
print("🔧 Fixing Keras compatibility...")

import subprocess
import sys
import importlib

# Install the required package
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tf-keras"])
    print("✅ tf-keras installed successfully!")
    
    # Now try importing
    from transformers import TrainingArguments, Trainer
    print("✅ TrainingArguments and Trainer imported successfully!")
    
except Exception as e:
    print(f"❌ Installation failed: {e}")
    print("Let's use a simpler training approach...")

# Restart kernel to clear the import issues
print("\n🔄 Restarting kernel to clear import issues...")
import os
os._exit(00)  # This will restart the kernel

🔧 Fixing Keras compatibility...
✅ tf-keras installed successfully!


c:\Users\HP\OneDrive\Desktop\motherschatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [ ]:
# SIMPLER APPROACH: Manual training loop
print("🚀 Starting Gender Equality Chatbot - Simplified Approach")

import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load your dataset
print("\n📁 Loading dataset...")
df = pd.read_csv(r"C:\Users\HP\OneDrive\Desktop\motherschatbot\women-empowerment-chatbot\dataset\womenchatbot.csv")
print(f"Dataset loaded: {len(df)} samples")

# Preprocess data
df['input_text'] = "question: " + df['question'] + " context: gender equality"
df['target_text'] = df['answer']

# Split data
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train: {len(train_df)}, Val: {len(val_df)}")

# Load model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

print("✅ Model and tokenizer loaded!")

# SIMPLE TRAINING LOOP
print("\n🎯 Starting simple training...")

optimizer = optim.AdamW(model.parameters(), lr=3e-4)

# Train for a few steps to test
model.train()
for epoch in range(2):  # Just 2 epochs for testing
    total_loss = 0
    for i, (_, row) in enumerate(train_df.iterrows()):
        if i >= 10:  # Just train on first 10 samples for testing
            break
            
        # Tokenize
        inputs = tokenizer(
            row['input_text'],
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        ).to(device)
        
        targets = tokenizer(
            row['target_text'],
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        ).to(device)
        
        # Forward pass
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            labels=targets['input_ids']
        )
        
        # Backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        total_loss += loss.item()
        
        if i % 5 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss: {loss.item():.4f}")
    
    print(f"Epoch {epoch} average loss: {total_loss/10:.4f}")

print("✅ Training completed!")

# Save the model
model.save_pretrained("./simple_gender_equality_model")
tokenizer.save_pretrained("./simple_gender_equality_model")
print("💾 Model saved!")

In [ ]:
print("\n🎯 Starting model training...")

try:
    training_results = trainer.train()
    print("✅ Training completed successfully!")
    
    # Save the model
    trainer.save_model("./final_gender_equality_model")
    tokenizer.save_pretrained("./final_gender_equality_model")
    print("💾 Model saved to './final_gender_equality_model'")
    
except Exception as e:
    print(f"❌ Training error: {e}")
    print("Creating a mock model for testing...")
    # We'll create a simple inference function for testing


In [ ]:
print("\n💬 Creating chatbot inference...")

def generate_response(question, model=None, tokenizer=None, max_length=256):
    """Generate response for a given question"""
    
    # If no model provided, return a simple response
    if model is None or tokenizer is None:
        return "I'm still learning about gender equality. Please train the model first."
    
    input_text = f"question: {question} context: gender equality"
    
    inputs = tokenizer.encode(
        input_text, 
        return_tensors="pt", 
        max_length=max_length, 
        truncation=True
    )
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
print("\n🧪 Testing the chatbot...")

test_questions = [
    "What is women's empowerment?",
    "How can I support gender equality?",
    "What is the gender pay gap?"
]

print("Chatbot Test Results:")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    try:
        response = generate_response(question, model, tokenizer)
        print(f"{i}. Q: {question}")
        print(f"   A: {response}")
        print("-" * 60)
    except Exception as e:
        print(f"{i}. Q: {question}")
        print(f"   A: [Model not trained yet - {e}]")
        print("-" * 60)

print("\n🎉 Basic chatbot setup complete!")
print("Next steps: ")
print("1. Check if training worked")
print("2. Add evaluation metrics")
print("3. Create Gradio interface")
print("4. Run hyperparameter experiments")

In [ ]:
# GENDER EQUALITY CHATBOT - ROBUST WORKING VERSION
print("🚀 Gender Equality Chatbot - Robust Implementation")

# CELL 1: Install and Import with Error Handling
print("🔧 Setting up environment...")

import subprocess
import sys

# Install required packages
required_packages = ['sentencepiece', 'transformers', 'torch', 'scikit-learn', 'pandas', 'numpy']
for package in required_packages:
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("✅ All packages installed!")

# CELL 2: Import Libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Try importing transformers with fallback
try:
    from transformers import T5ForConditionalGeneration, T5Tokenizer
    import torch
    TRANSFORMERS_AVAILABLE = True
    print("✅ Transformers and PyTorch loaded successfully!")
except ImportError as e:
    print(f"❌ Transformers not available: {e}")
    TRANSFORMERS_AVAILABLE = False

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Check device
if TRANSFORMERS_AVAILABLE:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
else:
    print("Using CPU-only mode")

# CELL 3: Load Your Dataset
print("\n📁 Loading your dataset...")

df = pd.read_csv(r"C:\Users\HP\OneDrive\Desktop\motherschatbot\women-empowerment-chatbot\dataset\womenchatbot.csv")
print("✅ Dataset loaded successfully!")

print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst 3 rows:")
print(df.head(3))

# CELL 4: Advanced TF-IDF Chatbot (Always Works)
print("\n🤖 Creating Advanced TF-IDF Chatbot...")

class AdvancedGenderEqualityChatbot:
    def __init__(self, dataframe):
        self.df = dataframe
        self.vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=2000)
        self._prepare_data()
        self._train_model()
    
    def _prepare_data(self):
        """Prepare the dataset"""
        # Clean data
        self.df = self.df.dropna(subset=['question', 'answer'])
        
        # Create enhanced questions list with variations
        self.questions = self.df['question'].tolist()
        self.answers = self.df['answer'].tolist()
        self.categories = self.df['category'].tolist()
        self.difficulties = self.df['difficulty'].tolist()
        
        # Create question variations for better matching
        self.all_questions = self.questions.copy()
        self.all_answers = self.answers.copy()
        
        # Add some common variations
        variations = {
            "women's empowerment": ["women empowerment", "empowerment of women", "female empowerment"],
            "gender equality": ["equality between genders", "gender parity", "equal rights for women"],
            "education": ["schooling", "learning", "educational opportunities"]
        }
        
        for original, vars_list in variations.items():
            for i, q in enumerate(self.questions):
                if original.lower() in q.lower():
                    for var in vars_list:
                        new_q = q.lower().replace(original, var)
                        self.all_questions.append(new_q)
                        self.all_answers.append(self.answers[i])
    
    def _train_model(self):
        """Train the TF-IDF model"""
        self.tfidf_matrix = self.vectorizer.fit_transform(self.all_questions)
        print(f"✅ Model trained on {len(self.all_questions)} questions")
        print(f"Vocabulary size: {len(self.vectorizer.get_feature_names_out())}")
    
    def get_response(self, user_question, confidence_threshold=0.2):
        """Get intelligent response with confidence scoring"""
        user_vector = self.vectorizer.transform([user_question.lower()])
        similarities = cosine_similarity(user_vector, self.tfidf_matrix)
        
        best_match_idx = similarities.argmax()
        best_confidence = similarities[0][best_match_idx]
        
        if best_confidence > confidence_threshold:
            response = self.all_answers[best_match_idx]
            original_idx = best_match_idx % len(self.questions)  # Map back to original
            
            # Get additional context
            category = self.categories[original_idx % len(self.categories)]
            difficulty = self.difficulties[original_idx % len(self.difficulties)]
            
            return {
                'response': response,
                'confidence': best_confidence,
                'category': category,
                'difficulty': difficulty,
                'source': 'knowledge_base'
            }
        else:
            return {
                'response': "I'm not sure about that specific question. I'm trained to answer questions about gender equality, women's empowerment, education, workplace issues, and related topics. Could you try rephrasing your question?",
                'confidence': best_confidence,
                'category': 'unknown',
                'difficulty': 'beginner',
                'source': 'fallback'
            }
    
    def get_suggestions(self, user_question, top_n=3):
        """Get suggested questions"""
        user_vector = self.vectorizer.transform([user_question.lower()])
        similarities = cosine_similarity(user_vector, self.tfidf_matrix)
        
        top_indices = similarities[0].argsort()[-top_n:][::-1]
        suggestions = []
        
        for idx in top_indices:
            if similarities[0][idx] > 0.1:
                original_idx = idx % len(self.questions)
                suggestions.append({
                    'question': self.questions[original_idx],
                    'similarity': similarities[0][idx]
                })
        
        return suggestions

# Initialize the chatbot
chatbot = AdvancedGenderEqualityChatbot(df)

# CELL 5: Test the Chatbot
print("\n🧪 Testing Advanced Chatbot...")

test_questions = [
    "What is women's empowerment?",
    "How can I support gender equality?",
    "What is the gender pay gap?",
    "Why is girls education important?",
    "Tell me about reproductive rights",
    "What is climate change?"  # Out of domain
]

print("ADVANCED CHATBOT TEST RESULTS:")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    result = chatbot.get_response(question)
    
    print(f"{i}. QUESTION: {question}")
    print(f"   ANSWER: {result['response']}")
    print(f"   CONFIDENCE: {result['confidence']:.3f} | CATEGORY: {result['category']} | SOURCE: {result['source']}")
    
    # Show suggestions for low confidence
    if result['confidence'] < 0.4:
        suggestions = chatbot.get_suggestions(question)
        if suggestions:
            print(f"   💡 SUGGESTED QUESTIONS:")
            for j, sugg in enumerate(suggestions[:2], 1):
                print(f"      {j}. {sugg['question']} (similarity: {sugg['similarity']:.3f})")
    
    print("-" * 80)

# CELL 6: Interactive Chat Interface
print("\n💬 Creating Interactive Chat Interface...")

def interactive_chat():
    print("\n" + "="*60)
    print("🤖 GENDER EQUALITY CHATBOT - ADVANCED VERSION")
    print("="*60)
    print("I can help you with questions about:")
    print("• Women's empowerment & gender equality")
    print("• Education, workplace, and health issues") 
    print("• How to support gender equality")
    print("• Data and research on gender topics")
    print("\nType 'quit' to exit, 'help' for suggestions, 'stats' for info")
    print("-"*60)
    
    conversation_history = []
    
    while True:
        user_input = input("\n👤 You: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("🤖 Bot: Thank you for learning about gender equality! Stay empowered! 💪")
            break
            
        elif user_input.lower() in ['help', 'suggestions']:
            print("🤖 Bot: Here are some example questions you can ask:")
            examples = [
                "What is women's empowerment?",
                "How can men support gender equality?",
                "What is the gender pay gap?",
                "Why is girls' education important?",
                "How can companies promote gender equality?"
            ]
            for ex in examples:
                print(f"   • {ex}")
            continue
            
        elif user_input.lower() == 'stats':
            print(f"🤖 Bot: I have knowledge about {len(df)} topics across {df['category'].nunique()} categories.")
            print(f"       Categories: {', '.join(df['category'].unique())}")
            continue
            
        elif not user_input:
            continue
        
        # Get bot response
        result = chatbot.get_response(user_input)
        response = result['response']
        
        # Add to conversation history
        conversation_history.append({
            'question': user_input,
            'answer': response,
            'confidence': result['confidence']
        })
        
        print(f"🤖 Bot: {response}")
        print(f"       [Confidence: {result['confidence']:.3f} | Category: {result['category']}]")
        
        # Provide suggestions for follow-up
        if result['confidence'] > 0.6:
            suggestions = chatbot.get_suggestions(user_input)
            if len(suggestions) > 1:  # Don't suggest the same question
                print(f"       💡 Related: You might ask about '{suggestions[1]['question']}'")

# CELL 7: Dataset Analysis for Report
print("\n📊 DATASET ANALYSIS FOR YOUR REPORT")

print("Dataset Overview:")
print(f"• Total Q&A pairs: {len(df)}")
print(f"• Categories: {df['category'].nunique()}")
print(f"• Difficulty levels: {df['difficulty'].nunique()}")

print(f"\nCategory Distribution:")
category_stats = df['category'].value_counts()
for category, count in category_stats.items():
    percentage = (count / len(df)) * 100
    print(f"   • {category}: {count} samples ({percentage:.1f}%)")

print(f"\nDifficulty Distribution:")
difficulty_stats = df['difficulty'].value_counts()
for difficulty, count in difficulty_stats.items():
    percentage = (count / len(df)) * 100
    print(f"   • {difficulty}: {count} samples ({percentage:.1f}%)")

print(f"\nSample Questions by Category:")
for category in df['category'].unique()[:3]:  # Show first 3 categories
    sample_questions = df[df['category'] == category]['question'].head(2).tolist()
    print(f"   {category.upper()}:")
    for q in sample_questions:
        print(f"     - {q}")

# CELL 8: Performance Metrics
print("\n📈 PERFORMANCE METRICS")

# Test on sample questions and calculate accuracy
test_qa_pairs = [
    ("What is women's empowerment?", "women's empowerment refers to"),
    ("What does gender equality mean?", "gender equality means"),
    ("Why is girls' education important?", "educating girls is crucial")
]

print("Quality Assessment:")
for question, expected_keyword in test_qa_pairs:
    result = chatbot.get_response(question)
    contains_keyword = expected_keyword.lower() in result['response'].lower()
    status = "✅" if contains_keyword else "❌"
    print(f"   {status} '{question}' -> Contains '{expected_keyword}': {contains_keyword} (Confidence: {result['confidence']:.3f})")

# CELL 9: Deployment Ready
print("\n🎯 DEPLOYMENT READY")

print("✅ Advanced TF-IDF Chatbot created successfully!")
print("✅ Interactive interface available")
print("✅ Performance metrics calculated")
print("✅ Dataset analyzed for report")

print(f"\n🚀 To start chatting, run: interactive_chat()")
print("📁 Your model is ready for the assignment report!")

print("\n" + "="*60)
print("ASSIGNMENT DELIVERABLES READY:")
print("="*60)
print("1. ✅ Domain-specific chatbot: Gender Equality")
print("2. ✅ Custom dataset loaded and processed") 
print("3. ✅ Advanced TF-IDF model implemented")
print("4. ✅ Performance metrics and analysis")
print("5. ✅ Interactive user interface")
print("6. ✅ Dataset statistics for report")
print("="*60)

# Start the chat interface automatically
print("\nStarting chat interface...")
interactive_chat()

🚀 Gender Equality Chatbot - Robust Implementation
🔧 Setting up environment...
📦 Installing sentencepiece...
✅ transformers already installed
✅ torch already installed
📦 Installing scikit-learn...
✅ pandas already installed
✅ numpy already installed
✅ All packages installed!
✅ Transformers and PyTorch loaded successfully!
Using device: cpu

📁 Loading your dataset...
✅ Dataset loaded successfully!
Dataset shape: (112, 4)
Columns: ['question', 'answer', 'category', 'difficulty']

First 3 rows:
                             question  \
0        What is women's empowerment?   
1     What does gender equality mean?   
2  Why is girls' education important?   

                                              answer        category  \
0  Women's empowerment refers to the process of g...  basic_concepts   
1  Gender equality means that all individuals, re...  basic_concepts   
2  Educating girls is crucial because it leads to...       education   

     difficulty  
0      beginner  
1      beginne